In [11]:
import os
import sys
import sqlite3
import numpy as np
import pandas as pd
import geopandas as gp
import plotly.express as px
import matplotlib.pyplot as plt
import scipy.stats as scs

import pyet
import pyeto
import spei

In [12]:
sys.path.append('/home/pooya/w/DroughtMonitoringIran/')

DATABASE_PATH = "./database/database.db"

### Monthly Data

In [13]:
conn = sqlite3.connect(DATABASE_PATH)

ground_data = pd.read_sql(sql='SELECT * FROM ground_data_monthly', con=conn)
ground_data['Date'] = pd.to_datetime(ground_data['Date'])
ground_data["Date"] = ground_data["Date"].dt.to_period("M").astype(str)
ground_data['Station_ID'] = ground_data['Station_ID'].astype(str)

ground_pet_data = pd.read_sql(sql='SELECT * FROM ground_pet_monthly', con=conn)
ground_pet_data['Date'] = pd.to_datetime(ground_pet_data['Date'])
ground_pet_data["Date"] = ground_pet_data["Date"].dt.to_period("M").astype(str)
ground_pet_data['Station_ID'] = ground_pet_data['Station_ID'].astype(str)


ground_di_data = pd.read_sql(sql='SELECT * FROM ground_di_monthly', con=conn)
ground_di_data['Date'] = pd.to_datetime(ground_di_data['Date'])
ground_di_data["Date"] = ground_di_data["Date"].dt.to_period("M").astype(str)
ground_di_data['Station_ID'] = ground_di_data['Station_ID'].astype(str)


gee_precip_data = pd.read_sql(sql='SELECT * FROM gee_precip_monthly', con=conn)
gee_precip_data['Date'] = pd.to_datetime(gee_precip_data['Date'])
gee_precip_data["Date"] = gee_precip_data["Date"].dt.to_period("M").astype(str)
gee_precip_data['Station_ID'] = gee_precip_data['Station_ID'].astype(str)


gee_pet_data = pd.read_sql(sql='SELECT * FROM gee_pet_monthly', con=conn)
gee_pet_data['Date'] = pd.to_datetime(gee_pet_data['Date'])
gee_pet_data["Date"] = gee_pet_data["Date"].dt.to_period("M").astype(str)
gee_pet_data['Station_ID'] = gee_pet_data['Station_ID'].astype(str)


gee_indices_data = pd.read_sql(sql='SELECT * FROM gee_indices_monthly', con=conn)
gee_indices_data['Date'] = pd.to_datetime(gee_indices_data['Date'])
gee_indices_data["Date"] = gee_indices_data["Date"].dt.to_period("M").astype(str)
gee_indices_data['Station_ID'] = gee_indices_data['Station_ID'].astype(str)


gee_di_data = pd.read_sql(sql='SELECT * FROM gee_di_monthly', con=conn)
gee_di_data['Date'] = pd.to_datetime(gee_di_data['Date'])
gee_di_data["Date"] = gee_di_data["Date"].dt.to_period("M").astype(str)
gee_di_data['Station_ID'] = gee_di_data['Station_ID'].astype(str)


ground_geoinfo = pd.read_sql(sql='SELECT * FROM ground_data_geoinfo', con=conn)
ground_geoinfo['Station_ID'] = ground_geoinfo['Station_ID'].astype(str)


conn.close()

In [14]:
data = ground_geoinfo.copy()

data = data\
    .merge(ground_data, on='Station_ID', how='outer')\
    .merge(ground_pet_data, on=['Station_ID', 'Date'], how='outer')\
    .merge(ground_di_data, on=['Station_ID', 'Date'], how='outer')\
    .merge(gee_precip_data, on=['Station_ID', 'Date'], how='outer')\
    .merge(gee_pet_data, on=['Station_ID', 'Date'], how='outer')\
    .merge(gee_indices_data, on=['Station_ID', 'Date'], how='outer')\
    .merge(gee_di_data, on=['Station_ID', 'Date'], how='outer')\
    .query('Date >= "2006-09" and Date <= "2024-06"')\
    .sort_values(['Station_ID', 'Date'])\
    .reset_index(drop=True)

data

,Station_ID,Station_Name,Province,Station_Latitude,Station_Longitude,Station_Elevation,Date,Temperature_Maximum,Temperature_Minimum,Temperature,...,ERA5_SPI_24,ERA5_SPEI_24,GPM_SPI_24,GPM_SPEI_24,TRMM_SPI_24,TRMM_SPEI_24,TERRACLIMATE_SPI_24,TERRACLIMATE_SPEI_24,PERSIANNCDR_SPI_24,PERSIANNCDR_SPEI_24
0,40709,Astara,Gilan,38.365,48.855,-21.1,2006-09,25.6,18.4,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40709,Astara,Gilan,38.365,48.855,-21.1,2006-10,22.2,16.7,19.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,40709,Astara,Gilan,38.365,48.855,-21.1,2006-11,16.6,8.2,12.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40709,Astara,Gilan,38.365,48.855,-21.1,2006-12,10.3,2.5,6.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,40709,Astara,Gilan,38.365,48.855,-21.1,2007-01,12.8,2.4,7.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5131,99361,Alasht,Mazandaran,36.071,52.843,1805.0,2024-02,7.5,-1.3,3.1,...,-1.494781,NaN,-0.884397,NaN,NaN,NaN,NaN,NaN,-5.877000,NaN
5132,99361,Alasht,Mazandaran,36.071,52.843,1805.0,2024-03,12.1,1.8,6.9,...,-1.630171,NaN,-1.943880,NaN,NaN,NaN,NaN,NaN,-6.136069,NaN
5133,99361,Alasht,Mazandaran,36.071,52.843,1805.0,2024-04,15.7,6.2,10.9,...,-1.800329,NaN,-1.772096,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5134,99361,Alasht,Mazandaran,36.071,52.843,1805.0,2024-05,18.0,9.3,13.6,...,-1.602138,NaN,-0.694518,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
data = data.round(3)
data

,Station_ID,Station_Name,Province,Station_Latitude,Station_Longitude,Station_Elevation,Date,Temperature_Maximum,Temperature_Minimum,Temperature,...,ERA5_SPI_24,ERA5_SPEI_24,GPM_SPI_24,GPM_SPEI_24,TRMM_SPI_24,TRMM_SPEI_24,TERRACLIMATE_SPI_24,TERRACLIMATE_SPEI_24,PERSIANNCDR_SPI_24,PERSIANNCDR_SPEI_24
0,40709,Astara,Gilan,38.365,48.855,-21.1,2006-09,25.6,18.4,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40709,Astara,Gilan,38.365,48.855,-21.1,2006-10,22.2,16.7,19.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,40709,Astara,Gilan,38.365,48.855,-21.1,2006-11,16.6,8.2,12.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40709,Astara,Gilan,38.365,48.855,-21.1,2006-12,10.3,2.5,6.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,40709,Astara,Gilan,38.365,48.855,-21.1,2007-01,12.8,2.4,7.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5131,99361,Alasht,Mazandaran,36.071,52.843,1805.0,2024-02,7.5,-1.3,3.1,...,-1.495,NaN,-0.884,NaN,NaN,NaN,NaN,NaN,-5.877,NaN
5132,99361,Alasht,Mazandaran,36.071,52.843,1805.0,2024-03,12.1,1.8,6.9,...,-1.630,NaN,-1.944,NaN,NaN,NaN,NaN,NaN,-6.136,NaN
5133,99361,Alasht,Mazandaran,36.071,52.843,1805.0,2024-04,15.7,6.2,10.9,...,-1.800,NaN,-1.772,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5134,99361,Alasht,Mazandaran,36.071,52.843,1805.0,2024-05,18.0,9.3,13.6,...,-1.602,NaN,-0.695,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5136 entries, 0 to 5135
Columns: 152 entries, Station_ID to PERSIANNCDR_SPEI_24
dtypes: float64(148), object(4)
memory usage: 6.0+ MB


In [16]:
conn = sqlite3.connect(DATABASE_PATH)

data.to_sql('data', conn, if_exists='replace', index=False)
conn.commit()

conn.close()